## Selector Group Chat

In [1]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("GROQ_API_KEY")

if api_key is None:
    raise ValueError("GROQ_API_KEY not found in environment variables")

model_client = OpenAIChatCompletionClient(
    api_key=api_key,
    base_url="https://api.groq.com/openai/v1",
    # model="meta-llama/llama-4-maverick-17b-128e-instruct",
    model="llama-3.1-8b-instant",
    model_info={
        "family": "llama",
        "vision": False,
        "function_calling": True,
        "json_output": True,
    },
) 

e:\Udemy Courses\Autogen_Crash_Course\.venv\Lib\site-packages\autogen_ext\models\openai\_openai_client.py:466: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [2]:
planning_agent = AssistantAgent(
    name = 'PlanningAgent',
    description= 'An agent for planning tasks,this agent should be the first to engage when given a new task.',
    model_client=model_client,
    system_message='''
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are :
        WebSearchAgent : Searches for information.
        DataAnalystAgent : Performs calculations

    You only plan and delegate tasks - you do not exectue them yourself.

    When assigning tasks, use the below format:
    1. <agent> : <task>

    After all the tasks are completed, summarize the findings and end with "TERMINATE"
    ''',
)

In [3]:
def search_web_tool(query:str)-> str:
    # Simulate a web search
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."

web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web_tool],
    system_message='''
        You are a web search agent.
        Your only tool is search_web_tool - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do calculations or data analysis on them.
    ''',
)

In [4]:
def percentage_change_tool(start:float, end:float) -> float:
    # Calculate percentage change
    if start == 0:
        return 0
    return ((end - start) / start) * 100


data_analyst_agent = AssistantAgent(
    name = 'DataAnalystAgent',
    description= 'An agent for performing calculations and data analysis.',
    model_client=model_client,
    tools= [percentage_change_tool],
    system_message='''
        You are a data analyst agent.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.

        If you have not seen the data, ask for it.

    ''',
)

In [5]:
from autogen_agentchat.conditions import MaxMessageTermination,TextMentionTermination

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=20)
combined_termination = text_mention_termination | max_messages_termination

In [6]:
selector_prompt = '''
Select an agent to perform the task.

{roles}

Current conversation history :
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planning agent has assigned task before other agents start working.
Only Select one agent.
'''

In [7]:
'''
Try not to overload the selector prompt with too much information.


selector_prompt (str, optional) – The prompt template to use for selecting the next speaker. 

Available fields: ‘{roles}’, ‘{participants}’, and ‘{history}’. 
1. {participants} is the names of candidates for selection. The format is [“<name1>”, “<name2>”, …]. 
2. {roles} is a newline-separated list of names and descriptions of the candidate agents. The format for each line is: “<name> : <description>”. 

3. {history} is the conversation history formatted as a double newline separated of names and message content. The format for each message is: “<name> : <message content>”.
'''

'\nTry not to overload the selector prompt with too much information.\n\n\nselector_prompt (str, optional) – The prompt template to use for selecting the next speaker. \n\nAvailable fields: ‘{roles}’, ‘{participants}’, and ‘{history}’. \n1. {participants} is the names of candidates for selection. The format is [“<name1>”, “<name2>”, …]. \n2. {roles} is a newline-separated list of names and descriptions of the candidate agents. The format for each line is: “<name> : <description>”. \n\n3. {history} is the conversation history formatted as a double newline separated of names and message content. The format for each message is: “<name> : <message content>”.\n'

In [8]:
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True)

In [9]:
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [ ]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='x3dp1c579', arguments='{"query":"Miami Heat player highest points 2006-2007 season"}', name='search_web_tool'), FunctionCall(id='td6j0h2g4', arguments='{"query":"Miami Heat player highest rebounds 2007-2008 season"}', name='search_web_tool'), FunctionCall(id='gf0zbsadj', arguments='{"query":"Miami Heat player name and rebounds 2008-2009 season"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 1397 points\n        James Posey: 550 points\n        ...\n        ', name='search_web_tool',

### Custom Selector Function

In [ ]:
s = "Mayank"
s[-1]

In [ ]:
planning_agent.name

In [ ]:
from typing import Sequence
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage

def selector_func(messages : Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if(messages[-1].source != planning_agent.name):
        return planning_agent.name
    return None

In [ ]:
await selector_team.reset()
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=selector_func)

In [ ]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))